# SarsCoV2 SRA data analysis with Pathogenie

## links

* https://www.ncbi.nlm.nih.gov/genbank/sars-cov-2-seqs/#sra-sequences
* https://www.ncbi.nlm.nih.gov/nuccore/NC_045512

In [41]:
import sys,os,shutil,subprocess
import glob
from importlib import reload
import numpy as np
import pandas as pd
import pylab as plt
from Bio import SeqIO
sys.path.append('pathogenie')
from pathogenie import app, tools, aligners

In [62]:
sra = pd.read_csv('sarscov2_sra.txt')
sra = sra.sort_values('Bytes')
print (sra['Assay Type'].value_counts())
sra = sra[sra['Assay Type']=='WGS']

AMPLICON    241
RNA-Seq      38
WGS           8
WGA           1
Name: Assay Type, dtype: int64


In [56]:
cols = ['Run','Collection_Date','Center Name','geo_loc_name_country','Bytes','Bases','Assay Type']
sra = sra.sort_values('Bytes')
sra.columns

Index(['Run', 'Assay Type', 'AvgSpotLen', 'Bases', 'BioProject', 'BioSample',
       'BioSampleModel', 'Bytes', 'Center Name', 'Consent',
       'DATASTORE filetype', 'DATASTORE provider', 'DATASTORE region',
       'Experiment', 'Instrument', 'Library Name', 'LibraryLayout',
       'LibrarySelection', 'LibrarySource', 'Organism', 'Platform',
       'ReleaseDate', 'Sample Name', 'SRA Study', 'Collection_Date', 'Host',
       'geo_loc_name_country', 'geo_loc_name_country_continent',
       'geo_loc_name', 'collected_by', 'Isolation_source', 'lat_lon',
       'host_disease', 'Isolate', 'strain', 'sample_type', 'Age',
       'Biomaterial_provider', 'host_description', 'host_tissue_sampled',
       'sex', 'tissue', 'AssemblyName', 'COUNTRY', 'env_broad_scale',
       'env_local_scale', 'env_medium', 'Host_Age', 'host_disease_outcome',
       'host_disease_stage', 'host_sex'],
      dtype='object')

In [58]:
sra[cols]

,Run,Collection_Date,Center Name,geo_loc_name_country,Bytes,Bases,Assay Type
119,SRR11140750,2020-02-14,UNIVERSITY OF WISCONSIN - MADISON,USA,3824341,7650166,WGS
115,SRR11140751,2020-02-14,UNIVERSITY OF WISCONSIN - MADISON,USA,21841904,23696161,WGS
117,SRR11140746,2020-02-14,UNIVERSITY OF WISCONSIN - MADISON,USA,77997568,159508570,WGS
118,SRR11140748,2020-02-14,UNIVERSITY OF WISCONSIN - MADISON,USA,92488120,190194752,WGS
116,SRR11140744,2020-02-14,UNIVERSITY OF WISCONSIN - MADISON,USA,108481864,226957916,WGS
112,SRR11140745,2020-02-14,UNIVERSITY OF WISCONSIN - MADISON,USA,240280298,260956833,WGS
114,SRR11140749,2020-02-14,UNIVERSITY OF WISCONSIN - MADISON,USA,280564726,304113442,WGS
113,SRR11140747,2020-02-14,UNIVERSITY OF WISCONSIN - MADISON,USA,332198738,359000999,WGS


In [ ]:
def fetch_sra_reads(df):
    #download a subset of reads
    path = 'sarscov2_fastq'
    for i,r in df.iterrows():        
        files = glob.glob(os.path.join(path,r.Run+'*'))    
        if len(files)==0:
            cmd = 'fastq-dump {n} --outdir {o}'.format(n=r.Run,o=path)
            print (cmd)
            subprocess.check_output(cmd,shell=True)
        
fetch_sra_reads(sra[:15])

In [60]:
path = 'sarscov2_fastq/'
files = app.get_files_from_paths([path])
df = app.get_samples(files)
df

,name,sample,filename,pair
0,SRR11140746,SRR11140746,sarscov2_fastq/SRR11140746.fastq.gz,1
1,SRR11140750,SRR11140750,sarscov2_fastq/SRR11140750.fastq.gz,1
2,SRR11140751,SRR11140751,sarscov2_fastq/SRR11140751.fastq.gz,1
